In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=e1918c04b02abb21a9e5ac08bedd9fa36d2fe4dc17b3a7723122fd877f766e55
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
import pandas as pd
import googletrans
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Read the dataset with a specific encoding
input_csv_file = 'spam.csv'  # Replace with the path of your dataset

# Try different encodings if necessary
df = pd.read_csv(input_csv_file, encoding='ISO-8859-1')

# Function to translate text to Telugu
def translate_to_telugu(text):
    try:
        translated = translator.translate(text, src='en', dest='te')
        return translated.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

# Translate only the first 500 rows of the 'v2' column
df_first_500 = df.head(500)  # Select the first 500 rows
df_first_500['v2'] = df_first_500['v2'].apply(translate_to_telugu)

# Save the updated DataFrame with translated 'v2' column to a new CSV
output_csv_file = 'telugu_translated_first_500_rows.csv'  # New output CSV file with Telugu text in 'v2'
df_first_500.to_csv(output_csv_file, index=False, encoding='utf-8')

print(f"CSV file with first 500 rows translated and saved as {output_csv_file}")


CSV file with first 500 rows translated and saved as telugu_translated_first_500_rows.csv


<ipython-input-3-ae1f6b8073b8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_500['v2'] = df_first_500['v2'].apply(translate_to_telugu)


In [ ]:
import pandas as pd

# ✅ Load CSV file with correct encoding
df = pd.read_csv("telugu_translated_first_500_rows.csv", encoding="utf-8-sig")

# ✅ Ensure only needed columns are present
df = df[['v1', 'v2']]  # Keeping only 'v1' and 'v2'

# ✅ Create an empty list to store modified rows
new_data = []

i = 0
while i < len(df):
    new_data.append(df.iloc[i])  # Add the current row

    if df.iloc[i]['v1'].lower() == 'spam':
        new_data.append(df.iloc[i])  # Duplicate the spam row

        if i + 1 < len(df) and df.iloc[i + 1]['v1'].lower() == 'ham':
            i += 1  # Skip the next ham row

    i += 1  # Move to the next row

# ✅ Convert list back to DataFrame
new_df = pd.DataFrame(new_data)

# ✅ Save the updated file
new_df.to_csv("balanced_translated.csv", index=False, encoding="utf-8-sig")

print("✅ Done! File saved as 'balanced_translated.csv'.")

✅ Done! File saved as 'balanced_translated.csv'.


In [ ]:
import pandas as pd
import requests

# ✅ Replace with your actual API key and region
API_KEY = "FrN1fqSKqGS60hCdVaCuy46V63oF7Kp5M8dz3EJ8q3eDrkzG4jVPJQQJ99BBACYeBjFXJ3w3AAAbACOGzZGM"  # <-- Paste your Key 1 here
LOCATION = "eastus"  # Example: "westeurope", "eastus", etc.

# Microsoft Translator API endpoint
ENDPOINT = "https://api.cognitive.microsofttranslator.com/translate"

# ✅ Load your CSV file with a different encoding
df = pd.read_csv("spam.csv", encoding='ISO-8859-1')  # Try 'ISO-8859-1' or 'utf-16'

# ✅ Select only the relevant columns (v1 for labels, v2 for sentences)
df = df[['v1', 'v2']]

# ✅ Function to translate text to Telugu
def translate_to_telugu(text):
    if pd.isna(text):  # Handle missing values
        return ""

    params = {
        "api-version": "3.0",
        "from": "en",
        "to": "te"  # Change 'hi' to 'te' for Telugu
    }

    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY,
        "Ocp-Apim-Subscription-Region": LOCATION,
        "Content-Type": "application/json"
    }

    body = [{"text": text}]

    response = requests.post(ENDPOINT, params=params, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()[0]["translations"][0]["text"]
    else:
        return f"Error: {response.status_code}"  # Handle API errors

# ✅ Translate only the first 501 rows in column 'v2' (the sentences) into Telugu
df.loc[:500, 'v2'] = df.loc[:500, 'v2'].apply(translate_to_telugu)

# ✅ Save only the first 501 rows (with both 'v1' and translated 'v2') into a new CSV file
df_first_501 = df.head(501)  # Get only the first 501 rows

df_first_501.to_csv("translated_dataset_telugu_first_501_rows.csv", index=False, encoding="utf-8-sig")

print("✅ Translation completed for the first 501 rows! File saved as 'translated_dataset_telugu_first_501_rows.csv'.")


✅ Translation completed for the first 501 rows! File saved as 'translated_dataset_telugu_first_501_rows.csv'.


In [ ]:
import pandas as pd

# ✅ Load CSV file with correct encoding
df = pd.read_csv("translated_dataset_telugu_first_501_rows.csv", encoding="utf-8-sig")

# ✅ Ensure only needed columns are present
df = df[['v1', 'v2']]  # Keeping only 'v1' and 'v2'

# ✅ Create an empty list to store modified rows
new_data = []

i = 0
while i < len(df):
    new_data.append(df.iloc[i])  # Add the current row

    if df.iloc[i]['v1'].lower() == 'spam':
        new_data.append(df.iloc[i])  # Duplicate the spam row

        if i + 1 < len(df) and df.iloc[i + 1]['v1'].lower() == 'ham':
            i += 1  # Skip the next ham row

    i += 1  # Move to the next row

# ✅ Convert list back to DataFrame
new_df = pd.DataFrame(new_data)

# ✅ Save the updated file
new_df.to_csv("Telugu_microsoft_balanced_translated.csv", index=False, encoding="utf-8-sig")

print("✅ Done! File saved as 'balanced_translated.csv'.")

✅ Done! File saved as 'balanced_translated.csv'.
